<div style="border-left: solid 6px #d6d6d6; padding: 10px; background-color: #eaeaea">
<b><i>Описание данных.</i></b></div>

Дано $\approx 15000$ собранных с торговой площадки лотов вторичной недвижимости (квартир) в европейском мегаполисе для того, чтобы с помощью методов моделирования и анализа данных определить факторы, значимые для цены этой недвижимости (цены предложения, которая выступает прокси переменной ценой сделки).

**Распределение количества комнат* и площади представлено ниже:**
\* После предобработки и чистки пропусков

* В выборке преобладают квартиры из 3 комнат, площадью 100 метров
* Медианное значение площади $75$ метров, 
* $1/10$ доля всех лотов не превышает 35 метров (`Студии и Апартаменты`), а $90\%$ лотов не превышают $150$ метров

<img src="../imgs/Число комнат и площади.png" alt="Цена за метр">

**Распределение цен предложения за единицу площади представлено ниже:**

<img src="../imgs/Цены за метр.png" alt="Цена за метр">

Также, посмотрим на площадь в зависимости от числа комнат.
* Интересно, что категория `Студии и Апартаменты` занимает наименьшую площадь, но распределение цен квадратного метра в них правее (значения больше), чем, например, в квартирах с `1 или даже 2 комнатами` (вероятно, причина в имеющейся выборке и другим характеристикам этой категории)

<img src="../imgs/Площадь и число комнат.png" alt="Цена за метр">

<img src="../imgs/Кол-во комнат и цена на метр (коробки).png" alt="Цена за метр">

* Удалим лоты без адреса (21 лот, не известна ни одна характеристика)
* Удалим лоты без этажа (1 лот с большой площадью в центре под офис)

* Заполним эвристиками (на основе распределений цен по известным) пропуски в поле `Кол-во комнат`:
    * **все** лоты меньше $31$ метров площади отнесем к категории `Студия или Апартаменты` (`СиА`), туда же отнесем недвижимость классифицированную дающим объявление, как `Аппартаменты`)
    * лоты с пропуском числа комнат и площадью до $42$ метров отнесем к категории `однокомнатные` (только $12%$ двухкомнатных меньше этого размера площади)
    * лоты с пропуском числа комнат и площадью до $70$ метров отнесем к категории `двухкомнатные` (только $20%$ трехкомнатных меньше этого размера площади)
    * лоты с пропуском числа комнат и площадью до $110$ метров отнесем к категории `трехкомнатные` (только $25%$ четырехкомнатных меньше этого размера площади)
    * лоты с пропуском числа комнат и площадью до $140$ метров отнесем к категории `четырехкомнатные` (только $25%$ пятикомнатных меньше этого размера площади)
    * остальные лоты с пропусками числа комнат отнесем к пятикомнатным
    * **все** лоты площадью больше $190$ метров отнесем к категории `Многокомнатная или свободная планировка` (`МиСП`)
    
* Заменим указание на еще не открывшиеся станции метро (`откроется в X`) в поле с расстоянием до метро на `NaN` (11 лотов)

<div style="border-left: solid 6px #d6d6d6; padding: 10px; background-color: #eaeaea">
<b><i>Преодобработки.</i></b></div>

> * Удалим пропуски которые нельзя восстановить
* Заполним эвристиками (на основе распределений цен по известным) пропуски в поле `Кол-во комнат`:
    * **все** лоты меньше $31$ метров площади отнесем к категории `Студия или Апартаменты` (`СиА`), туда же отнесем недвижимость классифицированную дающим объявление, как `Аппартаменты`)
    * **все** лоты площадью больше $190$ метров отнесем к категории `Многокомнатная или свободная планировка` (`МиСП`)
    * Остальные лоты с пропусками количества комнат заполним на **основании квантилей распределений площадей лотов с известных количеством комнат, с ограничением квантилями лотов большего числа комнат**
* Заменим указание на еще не открывшиеся станции метро (`откроется в X`) в поле с расстоянием до метро на `NaN` (11 лотов)
* Добавим признаки по образцу из статьи \[[ссылка](https://sun9-40.userapi.com/c857628/v857628789/1f0c46/eSwQt6LHzSg.jpg])\]
* Приведём типы (например, категориальные признаки, 'Округ', 'Район', 'Кол-во комнат', 'Метро', 'Тип дома', 'Окна', 'Ремонт'),  к категориями) и наивно закодируем категории (например, _р-н Гольяново_ просто станет $159$, без учета его связей/интеракци с другими признаками)

<div style="border-left: solid 6px #d6d6d6; padding: 10px; background-color: #eaeaea">
<b><i>Описание модели, метрик и валидации.</i></b></div>

> * Мы взяли алгоритм градиентного бустинга (имплементация lightgbm) и решали задачу регрессии
* Параметры подбирались интуитивно, с последующей валидацие на отложенной выборке с большей регуляризацией
* Основная модель валидации: отложенная выборка $50%$, со стратификацией по кол-ву комнат.
    * То есть, мы моделируем ситуацию, когда у нас было $\approx 7000$ лотов, а потом пришло еще 7000 тысяч, с примерным сохранением количества комнат*, и мы хотим узнать, насколько хорошо наша модель может объяснить цену на них.
    * Проблемой такого подхода может быть недостаточное количество наблюдений по категориям чтобы построить хорошую модель на них. Например, всего лотов _категории Z_ меньше $1000$, и в них есть высокая неоднородность
* С точки зрения метрик качества, в зависимости от степени регуляризации модели и категорий: 
    * $R^2$ лежит в интервале от $0.75$ до $0.9$ (доля объясненной дисперсии) 
    * относительные ошибки прогнозов лежат в интервале от $0.08$ до $0.3$ (от $\pm 8\%$ для студий до $\pm 30\%$ для неоднородных лотов $190+$ метров с отложенной выборкой в половину)

**Пример лога перед обучением:**
```
INFO:root:Моделируем: Цена за метр^2.
INFO:root:Не используем: ['Площадь', 'Площадь кухни'].
INFO:root:Используем: ['Округ', 'Район', 'Площадь лог', 'Кол-во комнат', 'Этаж', 'Этажей в доме', 'Первый этаж', 'Последний этаж', 'Метро', 'Расстояние до метро (ед)', 'Расстояние указано пешком', 'Расстояние указано на транспорте', 'Тип дома', 'Указан раздельный санузел', 'Несколько санузлов', 'Балкон указан', 'Окна', 'Окна во двор', 'Ремонт', 'Наличие ремонта', 'Возраст дома', 'Доля жилой площади', 'Доля куханной площади'].
```

<div style="border-left: solid 6px #d6d6d6; padding: 10px; background-color: #eaeaea">
<b><i>Описание предобработки.</i></b></div>

> * Приведём категориальные признаки колонки к категориями
* Закодируем категории